In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from egxpy.download import  get_OHLCV_data,get_EGX_intraday_data, get_EGXdata
from egxpy.optimization import portfolio_performance,optimize
import warnings

warnings.filterwarnings("ignore")
pio.templates.default='plotly_dark'

## Fetch close prices for Multiple Stocks

In [2]:
date = dt.today().date()
end = date
start = dt(2020, 5, 17).date()
daily_prices = get_EGXdata(stock_list=['ABUK','COMI','TMGH','FWRY', 'SWDY', 'TALM'],interval='Daily',start=start,end=end)
daily_prices    #daily close prices

you are using nologin method, data you access may be limited
ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


,ABUK,COMI,TMGH,FWRY,SWDY,TALM
Date,,,,,,
2020-05-17,11.93,32.175385,5.08,2.397528,7.51,NaN
2020-05-18,11.75,32.085648,5.00,2.381650,7.21,NaN
2020-05-19,11.87,32.379782,5.03,2.442892,7.28,NaN
2020-05-20,11.73,32.030809,5.01,2.565377,7.24,NaN
2020-05-21,11.96,31.163359,5.05,2.608474,7.43,NaN
...,...,...,...,...,...,...
2025-03-23,57.00,81.790000,53.53,9.300000,83.86,11.3
2025-03-24,55.74,81.900000,53.20,9.250000,81.70,11.0
2025-03-25,55.26,81.710000,53.09,9.200000,80.50,11.0


## Fetch open, high, low, close, volume for a single stock

In [3]:
ohlcv = get_OHLCV_data(symbol='ABUK',exchange='EGX',interval='Daily',n_bars=100)
ohlcv

,symbol,open,high,low,close,volume
datetime,,,,,,
2024-11-06 10:00:00,EGX:ABUK,55.88,56.35,55.84,55.90,402098.0
2024-11-07 10:00:00,EGX:ABUK,55.90,56.80,55.90,56.07,730040.0
2024-11-10 10:00:00,EGX:ABUK,56.07,56.39,55.60,55.81,1021359.0
2024-11-11 10:00:00,EGX:ABUK,55.81,55.83,55.04,55.20,2053266.0
2024-11-12 10:00:00,EGX:ABUK,55.20,55.38,54.65,55.01,1020655.0
...,...,...,...,...,...,...
2025-03-23 10:00:00,EGX:ABUK,57.08,57.50,56.53,57.00,254505.0
2025-03-24 10:00:00,EGX:ABUK,57.00,57.47,55.62,55.74,380735.0
2025-03-25 10:00:00,EGX:ABUK,55.74,56.00,55.05,55.26,255158.0


## Get intraday close prices at a desired timeframe 

In [4]:
start = end - timedelta(days=10) # last 10 days
intrday_prices = get_EGX_intraday_data(stock_list=['ABUK', 'COMI'], interval='5 Minute', start=start,end=end)
intrday_prices

,ABUK,COMI
datetime,,
2025-03-23 12:00:00+02:00,57.43,79.58
2025-03-23 12:05:00+02:00,57.40,79.57
2025-03-23 12:10:00+02:00,57.40,79.56
2025-03-23 12:15:00+02:00,57.45,79.52
2025-03-23 12:20:00+02:00,57.45,80.95
...,...,...
2025-03-27 14:55:00+02:00,56.90,82.05
2025-03-27 15:00:00+02:00,57.01,81.57
2025-03-27 15:05:00+02:00,57.00,81.53


In [12]:
optimal_weights = optimize(type='Sharpe',close=daily_prices,n=252,risk_free_rate=0.27,upper_bound=0.3)

In [6]:
optimal_weights

,ticker,weight
0,ABUK,0.1376
1,COMI,0.1931
2,TMGH,0.3000
3,FWRY,0.0000
4,SWDY,0.3000
5,TALM,0.0694


In [7]:
px.pie(optimal_weights, values='weight', names='ticker', title='Portfolio Weights')

In [8]:
mean_vector = daily_prices.pct_change().mean()
covariance_matrix = daily_prices.pct_change().cov().values

expected_returns, risk = portfolio_performance(optimal_weights['weight'],
                                               mean_returns=mean_vector,
                                               cov=covariance_matrix,n=252)
print(f'Expected Returns: {expected_returns:.2%}')
print(f'Risk: {risk:.2%}')


Expected Returns: 57.06%
Risk: 32.59%


In [9]:
w = optimal_weights['weight'].values.reshape((-1,1))

In [10]:
portfolio = daily_prices @ w

In [11]:
fig = go.Figure()

for stock in daily_prices.columns.to_list():
    fig.add_trace(go.Scatter(x=daily_prices.index,y=daily_prices[stock], name=stock))

fig.add_trace(go.Scatter(x=portfolio.index,y=portfolio.values.flatten(), name='Portoflio'))
fig.show()